In [1]:
%cd ..
%pwd

d:\ML Projects\Insurance_premium_predictor


'd:\\ML Projects\\Insurance_premium_predictor'

In [2]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

In [3]:
with ZipFile('Data\\IR_raw_data.zip', 'r') as file:
    file.extractall('Data\\')

In [4]:
data = pd.read_csv('Data\\IR_raw_data.csv', index_col='id')
data.shape

(1200000, 20)

- sampling 15k entries randomly or else my device is cooked 

In [5]:
data = data.sample(25000, random_state=13)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25000 entries, 549982 to 39887
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age                   24622 non-null  float64
 1   Gender                25000 non-null  object 
 2   Annual Income         24015 non-null  float64
 3   Marital Status        24595 non-null  object 
 4   Number of Dependents  22726 non-null  float64
 5   Education Level       25000 non-null  object 
 6   Occupation            17528 non-null  object 
 7   Health Score          23497 non-null  float64
 8   Location              25000 non-null  object 
 9   Policy Type           25000 non-null  object 
 10  Previous Claims       17505 non-null  float64
 11  Vehicle Age           24999 non-null  float64
 12  Credit Score          22108 non-null  float64
 13  Insurance Duration    25000 non-null  float64
 14  Policy Start Date     25000 non-null  object 
 15  Customer Feedback  

In [6]:
data.isnull().sum()

Age                      378
Gender                     0
Annual Income            985
Marital Status           405
Number of Dependents    2274
Education Level            0
Occupation              7472
Health Score            1503
Location                   0
Policy Type                0
Previous Claims         7495
Vehicle Age                1
Credit Score            2892
Insurance Duration         0
Policy Start Date          0
Customer Feedback       1599
Smoking Status             0
Exercise Frequency         0
Property Type              0
Premium Amount             0
dtype: int64

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [8]:
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

In [9]:
numerical_cols = data.select_dtypes(exclude='object').drop(['Premium Amount'], axis = 1).columns
categorical_cols = data.select_dtypes(include='object').columns

In [10]:
print(numerical_cols)
print(categorical_cols)

Index(['Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration'],
      dtype='object')
Index(['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location',
       'Policy Type', 'Policy Start Date', 'Customer Feedback',
       'Smoking Status', 'Exercise Frequency', 'Property Type'],
      dtype='object')


In [11]:
data[numerical_cols] = num_imputer.fit_transform(data[numerical_cols])
data[categorical_cols] = cat_imputer.fit_transform(data[categorical_cols])
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

In [12]:
data.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
549982,0.132251,Male,0.012510,Single,-1.483019,Master's,Unemployed,-0.503644,Urban,Comprehensive,-1.213954,1.106982,0.244894,0.752937,2023-10-06 15:21:39.231531,Average,No,Daily,Condo,29.0
661271,0.206482,Female,1.537974,Single,-0.003580,Bachelor's,Employed,-1.515212,Rural,Basic,1.203415,-0.962298,-0.975624,0.368432,2021-12-04 15:21:39.091080,Good,Yes,Weekly,House,40.0
95378,-0.610065,Female,-0.278826,Single,-0.003580,PhD,Employed,0.871427,Urban,Basic,1.203415,0.244782,1.768779,1.521946,2022-06-14 15:21:39.134960,Poor,No,Daily,House,3138.0
109118,-0.387370,Male,3.408589,Divorced,-1.483019,PhD,Self-Employed,-0.794070,Urban,Comprehensive,-0.005270,-0.272538,-1.067340,0.368432,2024-05-02 15:21:39.237118,Average,No,Monthly,Condo,1513.0
205036,1.468419,Male,1.217583,Single,-0.743300,Bachelor's,Employed,1.371232,Rural,Basic,-1.213954,-1.479618,0.019133,-1.169586,2020-03-05 15:21:39.146733,Poor,Yes,Monthly,Apartment,264.0


In [13]:
data['Policy Start Date'] = pd.to_datetime(data['Policy Start Date'])
data['Policy Age'] = (pd.to_datetime('2025-01-01') - data['Policy Start Date']).dt.days
data["Year"] = data["Policy Start Date"].dt.year.astype("float64")
data["Day"] = data["Policy Start Date"].dt.day.astype("float64")
data["Month"] = data["Policy Start Date"].dt.month.astype("float64")
data["Day_of_week"] = data["Policy Start Date"].dt.dayofweek.astype("float64")
data = data.drop(columns=['Policy Start Date'])

In [14]:
def cyclical_transform(df, col_name, max_val):
        # sin(2π * col / max_val), cos(2π * col / max_val)
        df[f"{col_name}_sin"] = np.sin(2 * np.pi * df[col_name] / max_val)
        df[f"{col_name}_cos"] = np.cos(2 * np.pi * df[col_name] / max_val)
        
cyclical_transform(data, "Month", 12)
cyclical_transform(data, "Day_of_week", 7)

data["Age"] = data["Age"] ** 2

In [15]:
for col in categorical_cols.drop('Policy Start Date'):
    print(data[col].value_counts(), '\n')

Gender
Male      12699
Female    12301
Name: count, dtype: int64 

Marital Status
Single      8679
Married     8241
Divorced    8080
Name: count, dtype: int64 

Education Level
Bachelor's     6328
PhD            6285
Master's       6261
High School    6126
Name: count, dtype: int64 

Occupation
Self-Employed    13545
Employed          5786
Unemployed        5669
Name: count, dtype: int64 

Location
Suburban    8441
Rural       8309
Urban       8250
Name: count, dtype: int64 

Policy Type
Premium          8494
Comprehensive    8272
Basic            8234
Name: count, dtype: int64 

Customer Feedback
Average    9479
Poor       7767
Good       7754
Name: count, dtype: int64 

Smoking Status
Yes    12568
No     12432
Name: count, dtype: int64 

Exercise Frequency
Weekly     6474
Rarely     6310
Monthly    6134
Daily      6082
Name: count, dtype: int64 

Property Type
Condo        8402
House        8361
Apartment    8237
Name: count, dtype: int64 



In [16]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

In [17]:
ord_cols = ['Policy Type','Customer Feedback','Exercise Frequency']
lab_cols = ['Gender','Marital Status','Education Level','Occupation','Location','Smoking Status','Property Type']

for col in ord_cols:
    data[col] = ordinal_encoder.fit_transform(data[col].to_frame())


In [18]:
data['Policy Age'] = scaler.fit_transform(data['Policy Age'].to_frame())

In [19]:
data = pd.get_dummies(data, columns=lab_cols, drop_first=True, dtype=float)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25000 entries, 549982 to 39887
Data columns (total 34 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Age                          25000 non-null  float64
 1   Annual Income                25000 non-null  float64
 2   Number of Dependents         25000 non-null  float64
 3   Health Score                 25000 non-null  float64
 4   Policy Type                  25000 non-null  float64
 5   Previous Claims              25000 non-null  float64
 6   Vehicle Age                  25000 non-null  float64
 7   Credit Score                 25000 non-null  float64
 8   Insurance Duration           25000 non-null  float64
 9   Customer Feedback            25000 non-null  float64
 10  Exercise Frequency           25000 non-null  float64
 11  Premium Amount               25000 non-null  float64
 12  Policy Age                   25000 non-null  float64
 13  Year            

In [21]:
data.to_csv('Data\\clean_data.csv', index=False)